In [0]:
%pip install requests_toolbelt
dbutils.library.restartPython()

In [0]:
import requests
import json
import uuid
from datetime import datetime
import os

In [0]:
dbutils.widgets.text("report_id", "")
report_id = dbutils.widgets.get("report_id")

dbutils.widgets.text("fight_id", "")
fight_id = dbutils.widgets.get("fight_id") or None

dbutils.widgets.text("data_type", "")
data_type = dbutils.widgets.get("data_type") # DamageDone or Buffs, Casts, Healing, etc.

def get_access_token():
    """
    Retrieves an OAuth2 access token from WarcraftLogs using client credentials stored in Databricks secrets.
    """
    client_id = dbutils.secrets.get(scope="warcraftlogs", key="client_id")
    client_secret = dbutils.secrets.get(scope="warcraftlogs", key="client_secret")

    response = requests.post(
        "https://www.warcraftlogs.com/oauth/token",
        data={
            "grant_type": "client_credentials",
            "client_id": client_id,
            "client_secret": client_secret
        }
    )
    return response.json()["access_token"]

token = get_access_token()
headers = {"Authorization": f"Bearer {token}"}

In [0]:
base_url = "https://www.warcraftlogs.com/api/v2/client"
start_time = None
page = 1

while True:
    # Build query arguments
    args = [f'dataType: {data_type}']
    if fight_id:
        args.append(f'fightIDs: [{fight_id}]')
    if start_time:
        args.append(f'startTime: {start_time}')
    arg_block = ", ".join(args)

    # Build GraphQL query
    query = f"""
    {{
      reportData {{
        report(code: "{report_id}") {{
          events({arg_block}) {{
            data
            nextPageTimestamp
          }}
        }}
      }}
    }}
    """

    # POST to the API
    response = requests.post(base_url, json={"query": query}, headers=headers)
    if response.status_code != 200:
        raise Exception(f"API call failed: {response.status_code} - {response.text}")

    json_data = response.json()
    
    # Error logging
    if "errors" in json_data:
        raise Exception(f"GraphQL Error: {json_data['errors']}")

    # Extract event data for this page
    try:
        events = json_data["data"]["reportData"]["report"]["events"]["data"]
    except KeyError:
        print(f"❌ Malformed response on page {page} — skipping")
        break

    # Skip saving if no events in this page
    if not events:
        print(f"⚠️ No events found on page {page}. Skipping save and ending pagination.")
        break

    # Save this page as JSON
    ts = datetime.utcnow().strftime("%Y%m%dT%H%M%S")
    filename = f"{report_id}_{fight_id or 'all'}_{data_type}_page{page}_{ts}.json"
    path = f"/Volumes/01_bronze/warcraftlogs/raw_api_calls/raw_events/{report_id}/{filename}"

    dbutils.fs.mkdirs(os.path.dirname(path))
    dbutils.fs.put(path, json.dumps(json_data), overwrite=True)
    print(f"✅ Saved page {page} to: {path}")

    # Prepare for next iteration
    try:
        start_time = json_data["data"]["reportData"]["report"]["events"]["nextPageTimestamp"]
        if start_time is None:
            break
    except KeyError:
        break

    page += 1
